<div align="left">
<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png" align="right">
<p align="left"><b>Authors: Mikel Baraza Vidal, Daniele Borghesi, Eva Cantín Larumbe, Francesco Pio Capoccello, Eva Teisberg</b></p>
</div>
<img align="left" width="30%" src="https://www.inf.upv.es/www/etsinf/wp-content/uploads/2017/02/ETSInf_PRINCIPAL_V-horizontal.png"/> <img width="30%" src="https://www.upv.es/perfiles/pas-pdi/imagenes/marca_UPV_principal_negro150.png" align="right"/>


In [671]:
#import folium
import requests
import json
#import geopandas as geo
from io import StringIO
import io
import pandas as pd
#------------------------------------

from ortools.linear_solver import pywraplp
from numpy import average, std
from statistics import *

###  Datasets

In [672]:
vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]

In [673]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

df_distances_stations = pd.DataFrame(distances_stations_db)
df_distances_stations =  pd.concat([df_distances_stations.iloc[-1:], df_distances_stations.iloc[:-1]], axis=0)
df_distances_stations.fillna(0, inplace=True)
df_distances_stations

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
42 South Street,0.00,228.56,282.07,367.00,299.82,397.01,523.64,609.91,444.76,511.11,...,1384.71,1348.97,1337.57,1553.31,2392.53,2406.78,2697.62,2833.72,2500.30,1567.00
49 Beekman Street,154.94,0.00,175.70,260.63,261.52,391.57,494.09,549.25,330.31,403.98,...,1401.76,1366.02,1354.62,1570.37,2409.58,2423.84,2609.88,2719.26,2396.89,1584.06
100 Duane Street,299.43,180.79,0.00,96.76,272.78,413.52,467.01,447.83,228.89,302.56,...,1411.62,1375.88,1364.48,1580.22,2419.44,2341.17,2469.12,2578.51,2256.14,1593.91
14 N. Moore Street,408.12,289.48,174.89,0.00,339.69,444.26,450.79,431.61,212.68,286.35,...,1520.31,1484.57,1473.17,1688.91,2405.59,2246.56,2374.51,2483.90,2161.52,1702.60
75 Canal Street,296.75,295.70,275.65,283.51,0.00,195.55,260.18,346.45,260.93,293.16,...,1563.49,1522.61,1516.35,1732.10,2571.31,2470.82,2598.77,2708.16,2385.79,1745.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345 Annadale Road,2387.88,2470.02,2355.43,2311.84,2446.72,2547.14,2553.66,2534.49,2315.55,2389.22,...,1315.83,1303.25,1115.74,877.46,478.78,0.00,300.33,537.43,414.36,1461.28
1560 Drumgoole Road West,2566.73,2569.97,2455.37,2411.78,2546.66,2647.08,2653.60,2634.43,2415.49,2489.16,...,1494.67,1482.10,1294.59,1056.30,560.66,220.62,0.00,411.10,284.85,1640.12
7219 Amboy Road,2891.38,2772.75,2658.16,2614.56,2749.45,2849.86,2856.39,2837.21,2618.27,2691.95,...,1847.28,1834.71,1647.20,1408.91,898.57,580.71,391.45,0.00,521.33,1992.73
1100 Rossville Ave,2425.57,2306.93,2192.34,2148.75,2283.63,2384.04,2390.57,2371.40,2152.46,2226.13,...,1408.77,1466.73,1208.69,1182.77,763.62,423.58,255.42,486.65,0.00,1554.22


In [674]:
distance = pd.read_csv("dataset/distance_stations_neighborhoods.csv", index_col=0)
neighborhood = pd.read_csv("dataset/neighborhoods_info_standardized.csv")
serviceability = {}

for vehicle_type in vehicle_types:
    serviceability[vehicle_type] = pd.read_csv("dataset/serviceability/" + vehicle_type +"_ntas_serviceability.csv", index_col=0)

In [675]:
distance.head()

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
QN08,2123.18,2130.20,2104.71,2073.11,1896.78,1881.48,1925.06,1876.57,2030.47,1974.10,...,2449.03,2393.86,2396.74,2617.64,3444.35,3471.11,3761.95,3933.27,3559.47,2631.33
BX28,1894.39,1874.63,1935.64,2020.57,1809.95,1751.58,1728.54,1601.29,1843.17,1762.19,...,3168.88,3133.14,3051.15,3274.89,3596.90,3272.39,3250.70,3276.83,3037.71,3351.18
QN55,1872.67,1776.72,1811.35,1821.87,1646.27,1630.97,1674.55,1626.81,1779.96,1724.34,...,2113.36,2058.18,2061.06,2281.96,3108.67,3135.43,3426.27,3597.59,3223.79,2295.65
BK50,1532.61,1587.44,1614.04,1630.87,1548.42,1533.11,1576.70,1662.97,1682.11,1704.08,...,1726.71,1671.54,1674.41,1895.32,2722.02,2748.79,3039.63,3210.95,2837.15,1909.01
BX41,1774.19,1754.43,1815.43,1900.37,1689.74,1631.38,1608.33,1481.08,1722.97,1641.99,...,3048.68,3012.94,2893.91,3117.65,3439.67,3115.16,3093.46,3119.59,2880.48,3201.09


In [676]:
neighborhood.head()

,nta,boro_name,name,shape_area,population,population_density,engine_needed_first_shift,engine_needed_second_shift,ladder_needed_first_shift,ladder_needed_second_shift,...,engine_needed_per_density_first_shift,engine_needed_per_density_second_shift,ladder_needed_per_density_first_shift,ladder_needed_per_density_second_shift,rescue_needed_per_density_first_shift,rescue_needed_per_density_second_shift,squad_needed_per_density_first_shift,squad_needed_per_density_second_shift,hazardous_needed_per_density_first_shift,hazardous_needed_per_density_second_shift
0,QN08,Queens,St. Albans,0.223361,0.367078,0.109918,0.292654,0.339234,0.278632,0.342927,...,0.108119,0.088128,0.106058,0.090719,0.100143,0.082782,0.090793,0.088288,0.097795,0.105349
1,BX28,Bronx,Van Cortlandt Village,0.062467,0.378462,0.341809,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,QN55,Queens,South Ozone Park,0.239059,0.573192,0.161128,0.487559,0.479621,0.464957,0.469801,...,0.169097,0.116969,0.166145,0.116673,0.173235,0.209827,0.193908,0.076507,0.183615,0.108229
3,BK50,Brooklyn,Canarsie,0.237903,0.632227,0.178528,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,BX41,Bronx,Mount Hope,0.028422,0.391357,0.616431,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Creating an instance of the Solver class

In [677]:
solver = pywraplp.Solver.CreateSolver('GLOP')

### Creating decision variables

with:

$X_{i, j, k}: $  number of vehicles of type ${i}$ deployed in station ${j}$ during shift ${k}$

$X_{i, j, k} \geq 0$, $X_{i, j, k} \in \mathbb{R}$

$ i \in I, j \in J, k \in K$

_________________

$Y_{i, a, b}:$  number of vehicles of type ${i}$ moved from station ${a}$ to station ${b}$ at shift change

$Y_{i, a, b} \geq 0$, $Y_{i, a, b} \in \mathbb{R}$

$ i \in I, a \in J, b \in J$

_____

$T_{i, j, k}: 1$ if a vehicle of type ${i}$ is deployed in station ${j}$ during shift ${k}$, $0$ otherwise

$T_{i, j, k} \geq 0$, $T_{i, j, k} \in \mathbb{Z}$

$ i \in I, j \in J, k \in K$

____


$Q_{j, k}: 1$ if there are more than two vehicles in station $j$ during shift $k$, $0$ otherwise

$Q_{j, k} \geq 0$, $Q_{j, k} \in \mathbb{Z}$

$ j \in J, k \in K$

____

- type of vehicles$: i=1\dots 5$

- stations: $j=1\dots 218$

- shifts: $k=1\dots 2$

- neighborhoods: $n=1 \dots 195$


In [678]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents = json.loads(response.text)

I = set()
for elem in incidents:
    I.update(elem['units'])

I = list(I)

In [679]:
N = list(neighborhood.nta)
J = list(distance.columns)
K= [1,2]

In [680]:
X = {}
for i in I:
    for j in J:
        for k in K:
            v = solver.NumVar(0, solver.infinity(), "Number of vehicles of type " + str(i) + " deployed at station "+ str(j)+ " during shift " + str(k))
            X[(i,j,k)] = v

In [681]:
Y = {}
for i in I:
    for a in J:
        for b in J:
            v = solver.NumVar(0, solver.infinity(), "Number of vehicles of type " + str(i) + " moved from station "+ str(a)+ " to station " + str(b))
            Y[(i,a,b)] = v

In [682]:
T = {}
for i in I:
    for j in J:
        for k in K:
            v = solver.BoolVar("if a vehicle of type" + str(i)  + " is deployed in station " + str(j) + " during shift " + str(k))
            T[(i,j,k)] = v

In [683]:
Q = {}
for j in J:
    for k in K:
        v = solver.BoolVar("if there are two or more vehicles in station " + str(j) + " during shift " + str(k))
        Q[(j,k)] = v

## Creting constraints

### 1. Station capacity

\begin{equation}
    \sum_{i \in I}^{}X_{i, j, k} \leq c_{j}, \forall j \in J, k \in K
    \label{station_capacity}
\end{equation}

In [684]:
response2 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/located_firehouses.json")
stations = json.loads(response2.text)

In [685]:
#capacities = [(feature['properties']['capacity'], feature['properties']['FacilityAddress']) for feature in stations['features']]

capacities = {}

for station in stations["features"]:
    station_address = station["properties"]["FacilityAddress"]
    capacities[station_address] = station["properties"]["capacity"]

In [686]:
station_capacity_constraints = []

for j in J:
    capacity = capacities[j]
    for k in K:
        c = solver.Constraint(-solver.infinity(), capacity)
    
        for i in I:
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)

        station_capacity_constraints.append(c)

assert len(station_capacity_constraints) == 438, "The number of constraints (" + str(len(station_capacity_constraints)) + ") is NOT correct"

### 2. Vehicles availability

\begin{equation}
    \sum_{j \in J}^{}X_{i, j, k} \leq v_{i}, \forall i \in I, k \in K
    \label{vehicle_availability}
\end{equation}

In [687]:
availabilities = {'engine': 197, 'ladder': 143, 'hazardous': 1, 'squad': 8, 'rescue':5}

In [688]:
vehicles_availability_constraints = []

for i in I:
    availability = availabilities[i]

    for k in K:
        c = solver.Constraint(-solver.infinity(), availability)

        for j in J:
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
            
        vehicles_availability_constraints.append(c)

assert len(vehicles_availability_constraints) == 10, "The number of constraints (" + str(len(vehicles_availability_constraints)) + ") is NOT correct"

### 3. At least one vehicle

\begin{equation}
    \sum_{i \in I}^{}X_{i, j, k} \geq 1, \forall j \in J, k \in K
    \label{at_least_one_vehicle}
\end{equation}

In [689]:
atleast_one_vehicle_constraints = []

for j in J:
    for k in K:
        c = solver.Constraint(1, solver.infinity())
    
        for i in I:
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
        atleast_one_vehicle_constraints.append(c)

assert len(atleast_one_vehicle_constraints) == 438, "The number of constraints (" + str(len(atleast_one_vehicle_constraints)) + ") is NOT correct"

### 4. Scarce resources concentration

In [690]:
scarce_resources_concentration_constraints = []

for k in K:
    for n in N:
        for i in {"rescue", "squad"}:
            c = solver.Constraint(1, 1)
            
            for j in J:
                s = 1 if serviceability[i].loc[n, j] == True else 0
                v = X[(i, j, k)]
                c.SetCoefficient(v, s)

            scarce_resources_concentration_constraints.append(c)

assert len(scarce_resources_concentration_constraints) == 780, "The number of constraints (" + str(len(scarce_resources_concentration_constraints)) + ") is NOT correct"

### 5. Area resources concentration

In [691]:
area_resources_concentration_constraints = []

for k in K:
    for n in N:
        for i in {"engine", "ladder"}:
            s_max = 0
            
            for j in J:
                s_max += 1 if serviceability[i].loc[n, j] == True else 0
            
            c = solver.Constraint(1, s_max)
            
            for j in J:
                s = 1 if serviceability[i].loc[n, j] == True else 0
                v = X[(i, j, k)]
                c.SetCoefficient(v, s)

            area_resources_concentration_constraints.append(c)
        
assert len(area_resources_concentration_constraints) == 780, "The number of constraints (" + str(len(area_resources_concentration_constraints)) + ") is NOT correct"

### 6. Station resources concentration

In [692]:
station_resources_concentration_constraints = []

for k in K:
    for j in J:
        for i in {"engine", "ladder"}:
            c = solver.Constraint(-solver.infinity(), 2)
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
            station_resources_concentration_constraints.append(c)
                    
assert len(station_resources_concentration_constraints) == 876, "The number of constraints (" + str(len(station_resources_concentration_constraints)) + ") is NOT correct"

### 7. At least two vehicle types

#### 7.1 Linking constrain T

In [693]:
T_linking_constraints = []

for i in I:
    for j in J:
        capacity = capacities[j]
        for k in K:
            v_T = T[(i, j, k)]
            v_X = X[(i, j, k)]

            c= solver.Constraint(0, 0)
            c.SetCoefficient(v_T, -capacity - 1)
            c.SetCoefficient(v_X, 1)

            T_linking_constraints.append(c)

assert len(T_linking_constraints) == 2190, "The number of constraints (" + str(len(T_linking_constraints)) + ") is NOT correct"

#### 7.2. Linking constraint Q

In [694]:
Q_linking_constraints = []

for j in J:
    capacity = capacities[j]
    for k in K:
        c = solver.Constraint(-solver.infinity(), 1)
        v_Q = Q[(j, k)]
        for i in I:
            v_X = X[(i, j, k)]

            c.SetCoefficient(v_X, 1)
        c.SetCoefficient(v_Q, -capacity)
        
        Q_linking_constraints.append(c)

assert len(Q_linking_constraints) == 438, "The number of constraints (" + str(len(Q_linking_constraints)) + ") is NOT correct"

In [695]:
atleast_two_types_of_vehicle_constraints = []

for j in J:
    capacity = capacities[j]
    for k in K:
        c = solver.Constraint(0, solver.infinity())
        v_Q = Q[(j, k)]
        for i in I:
            v_T = T[(i, j, k)]

            c.SetCoefficient(v_X, 1)
        c.SetCoefficient(v_Q, -2)
        atleast_two_types_of_vehicle_constraints.append(c)

assert len(atleast_two_types_of_vehicle_constraints) == 438, "The number of constraints (" + str(len(atleast_two_types_of_vehicle_constraints)) + ") is NOT correct"

### 8. Vehicles transfer

In [696]:
vehicles_transfer_constraints = []

for i in I:
    for a in J:
        c = solver.Constraint(0, 0)

        for j in J:
            Y_aj = Y[(i, a, j)]
            Y_ja = Y[(i, j, a)]

            c.SetCoefficient(Y_aj, 1)
            c.SetCoefficient(Y_ja, -1)
        
        X_a1 = X[(i, a, 1)]
        X_a2 = X[(i, a, 2)]

        c.SetCoefficient(X_a1, -1)
        c.SetCoefficient(X_a2, 1)

        vehicles_transfer_constraints.append(c)

assert len(vehicles_transfer_constraints) == 1095, "The number of constraints (" + str(len(vehicles_transfer_constraints)) + ") is NOT correct"

### 9. Maximum transfer distance

In [697]:
serviceability_distances = {
    "engine": 754,
    "ladder": 754,
    "rescue": 2660,
    "squad": 2530,
    "hazardous": 4000,
}

In [698]:
max_transfer_distance_constraints = []
a_counter = -1

for i in I:
    for a in J:
        a_counter += 1
        for b in J[a_counter:]:
            
            route_distance = df_distances_stations.loc[a, b]
            type_distance_limit = serviceability_distances[i]

            c = solver.Constraint(-solver.infinity(), 0)
            
            v = Y[(i, a, b)]

            c.SetCoefficient(v, route_distance - type_distance_limit)

            max_transfer_distance_constraints.append(c)

assert len(max_transfer_distance_constraints) == 24090, "The number of constraints (" + str(len(max_transfer_distance_constraints)) + ") is NOT correct"

### Objective Function


$max Z: \sum_{k=1}^{2} \sum_{i=1}^{5} \sum_{j=1}^{218} Y_{i,j,k} \left(
\sum_{i=1}^{195} \cdot s_{j,n,k,i} \cdot (0.4p_n + 0.1h_{n,i,k} - 0.5d_{n,j} \cdot j) \right)$